# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
weather_data= pd.read_csv("../output_data/weather_data.csv",encoding='utf-8', keep_default_na=False)
weather_data.head(15)

,city,lat,Lon,temp,humidity,Wind_speed,Cloudiness,Country_code,Date
0,kruisfontein,-34.00,24.73,23.33,67,4.92,51,ZA,1602857035
1,palenque,17.52,-91.97,24.93,84,0.65,67,MX,1602857035
2,mackay,-21.15,149.20,22.00,60,6.70,40,AU,1602856802
3,krasnoselkup,65.70,82.47,-1.37,99,5.06,100,RU,1602857035
4,camana,-16.62,-72.71,20.80,61,3.59,5,PE,1602857035
5,axim,4.87,-2.24,27.06,80,5.02,62,GH,1602857036
6,albany,42.60,-73.97,11.11,89,0.31,100,US,1602857036
7,kudahuvadhoo,2.67,72.89,28.41,77,9.46,100,MV,1602857036
8,tosya,41.02,34.04,23.00,28,1.00,0,TR,1602857036
9,victor harbor,-35.57,138.62,13.33,86,0.45,100,AU,1602857036


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
import gmaps

# Google developer API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [50]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(46.0, -5.0), zoom_level=2)

In [51]:
# Assign marker
locations = weather_data[['lat', 'Lon']]
weights = weather_data['humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,dissipating=False, max_intensity=100, point_radius=3))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
cleand_data = weather_data.loc[(weather_data["Wind_speed"] <= 10) & (weather_data["Cloudiness"] == 0) & (weather_data["temp"] <= 26.6) & (weather_data["temp"] >= 21.1)].dropna()
cleand_data.head(10)


# A max temperature lower than 80 degrees but higher than 70.I have converted this Farenheit to Celsius.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

,city,lat,Lon,temp,humidity,Wind_speed,Cloudiness,Country_code,Date
8,tosya,41.02,34.04,23.00,28,1.00,0,TR,1602857036
25,gualeguay,-33.14,-59.31,23.21,56,3.84,0,AR,1602857039
27,bredasdorp,-34.53,20.04,22.00,60,8.70,0,ZA,1602857039
41,saldanha,-33.01,17.94,25.00,38,5.70,0,ZA,1602857041
128,khuzdar,27.74,66.64,22.50,17,2.89,0,PK,1602857055
129,port hedland,-20.32,118.57,26.00,57,5.70,0,AU,1602857439
173,dajal,29.56,70.38,25.93,17,1.04,0,PK,1602857011
226,emerald,-23.53,148.17,22.00,53,4.10,0,AU,1602857070
256,guerrero negro,27.98,-114.06,21.74,70,4.05,0,MX,1602857075
300,narasannapeta,18.42,84.05,26.56,88,2.92,0,IN,1602856975


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_df =cleand_data.loc[:,['city','Country_code','lat','Lon']]
hotel_df["Hotel_name"] =""

hotel_df

,city,Country_code,lat,Lon,Hotel_name
8,tosya,TR,41.02,34.04,
25,gualeguay,AR,-33.14,-59.31,
27,bredasdorp,ZA,-34.53,20.04,
41,saldanha,ZA,-33.01,17.94,
128,khuzdar,PK,27.74,66.64,
129,port hedland,AU,-20.32,118.57,
173,dajal,PK,29.56,70.38,
226,emerald,AU,-23.53,148.17,
256,guerrero negro,MX,27.98,-114.06,
300,narasannapeta,IN,18.42,84.05,


In [72]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
     "radius": 5000, 
     "keyword":"hotel",
     "key": g_key,
}

In [74]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row["lat"]
    long = row["Lon"]
    city_name = row["city"]

    # add keyword to params dict
    params["location"] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index,"Hotel_name"] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    
print("######End#############")

Retrieving Results for Index 8: sambava.
Closest hotel in tosya is HD Miray Otel ve Restaurant.
------------
Retrieving Results for Index 25: sambava.
Closest hotel in gualeguay is Faro del Lago.
------------
Retrieving Results for Index 27: sambava.
Closest hotel in bredasdorp is De Hoop Collection.
------------
Retrieving Results for Index 41: sambava.
Closest hotel in saldanha is Protea Hotel by Marriott Saldanha Bay.
------------
Retrieving Results for Index 128: sambava.
Missing field/result... skipping.
------------
Retrieving Results for Index 129: sambava.
Closest hotel in port hedland is The Esplanade Hotel.
------------
Retrieving Results for Index 173: sambava.
Closest hotel in dajal is Jahan Serwer.
------------
Retrieving Results for Index 226: sambava.
Closest hotel in emerald is Emerald Star Hotel.
------------
Retrieving Results for Index 256: sambava.
Closest hotel in guerrero negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 300: sambava.
Closest

In [75]:
hotel_df

,city,Country_code,lat,Lon,Hotel_name
8,tosya,TR,41.02,34.04,HD Miray Otel ve Restaurant
25,gualeguay,AR,-33.14,-59.31,Faro del Lago
27,bredasdorp,ZA,-34.53,20.04,De Hoop Collection
41,saldanha,ZA,-33.01,17.94,Protea Hotel by Marriott Saldanha Bay
128,khuzdar,PK,27.74,66.64,
129,port hedland,AU,-20.32,118.57,The Esplanade Hotel
173,dajal,PK,29.56,70.38,Jahan Serwer
226,emerald,AU,-23.53,148.17,Emerald Star Hotel
256,guerrero negro,MX,27.98,-114.06,Hotel Los Caracoles
300,narasannapeta,IN,18.42,84.05,SPR Grand Hotel


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "Lon"]]

In [79]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))